In [1]:
import requests
from pandas.io.json import json_normalize
import dash
import dash_table
import dash_core_components as dcc
import dash_html_components as html
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.graph_objects as go
from dash.dependencies import Input, Output
from plotly.subplots import make_subplots


#Geo map of Cases per country globally
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import plotly as py
import plotly.graph_objs as go
init_notebook_mode(connected=True)


In [2]:
url = 'https://api.covid19api.com/summary'
r = requests.get(url)
dictr = r.json()
header = dictr['Countries']
df = json_normalize(header)
df = df.drop(df.index[[0,224]])
df = df.drop(columns = 'Slug')
df = df.sort_values(by=['TotalConfirmed'], ascending=False)
df.head()

/home/tshepo/.local/lib/python3.6/site-packages/ipykernel_launcher.py:5: FutureWarning:

pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead



,Country,CountryCode,NewConfirmed,TotalConfirmed,NewDeaths,TotalDeaths,NewRecovered,TotalRecovered,Date
234,United States of America,US,24513,604736,2227,25747,4281,47763,2020-04-15T21:27:04Z
206,Spain,ES,2442,172541,300,18056,2777,67504,2020-04-15T21:27:04Z
107,Italy,IT,2972,162488,602,21067,1695,37130,2020-04-15T21:27:04Z
80,Germany,DE,1287,131359,100,3294,3900,68200,2020-04-15T21:27:04Z
233,United Kingdom,GB,5275,94845,782,12129,19,323,2020-04-15T21:27:04Z


In [3]:
style_table = {
    'maxHeight': '50ex',
    'overflowY': 'scroll',
    'width': '30%',
    'minWidth': '30%',
}
style_cell = {
    'fontFamily': 'Open Sans',
    'textAlign': 'center',
    'height': '30px',
    'padding': '10px 22px',
    'whiteSpace': 'inherit',
    'overflow': 'hidden',
                'textOverflow': 'ellipsis',
}
style_cell_conditional = [
    {
        'if': {'column_id': 'State'},
        'textAlign': 'left'
    },
]
style_header = {
    'fontWeight': 'bold',
    'backgroundColor': "#3D9970",
    'fontSize': '16px'
}
style_data_conditional = [
    {
        'if': {'row_index': 'odd'},
        'backgroundColor': 'rgb(248, 248, 248)'
    }]
style_table = {
    'maxHeight': '50ex',
    'overflowY': 'scroll',
    'width': '100%',
    'minWidth': '100%',
}

In [4]:
a = df.TotalConfirmed.sum()-df.NewConfirmed.sum()

In [5]:
# # Style sheet
# external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']

# # App
# app = dash.Dash(__name__, external_stylesheets=external_stylesheets)

# app.layout = html.Div(children=[
#         #header
    
#     html.Div([
#         html.Div([
#             html.Img(src="https://dcassetcdn.com/design_img/3666062/781819/781819_21156990_3666062_e336b60a_image.png",className='four columns')
            
#         ], className="four columns"),
        

#         html.Div([
#             html.H3('Latest Covid 19 Update', style={
#                                 'fontFamily': 'Open Sans',
#                                 'textAlign': 'center',
#                             }),
             
#         ], className="four columns"),
        
#         html.Div([
#             html.H3('#StayAtHome', style={
#                                 'fontFamily': 'Open Sans',
#                                 'textAlign': 'right',
#                             }),
            
#         ], className="four columns")
#     ], className="row"),
    
#     dcc.Tabs(id='tabs-example', value='tab', children=[
#         dcc.Tab(label='Global Map', value='tab-1'),
#         dcc.Tab(label='South African cases', value='tab-2'),
#         dcc.Tab(label='Analysis', value='tab-3'),
#     ]),
#     html.Div(id='tabs-example-content')
# ])




# def render_content(tab):
#     '''Display the table and the graph sideby
#     side when the respective tab is selected'''

#     if tab == 'tab-1':
#         return html.Div([
#             html.Div([
       
#                 html.H3('band', style={
#                                     'fontFamily': 'Open Sans',
#                                     'textAlign': 'center',
#                                 })

#         ], className="row"),

#         #VISUALISATIONS
#          html.Div([
#             html.Div([
#                 html.H3('visualisation'),
#                 html.Div([
#                     html.Div(
#                         [
#                             html.P("Total confirmed cases"),
#                             html.H6(
#                                 id="well_text",
#                                 className="info_text"
#                             )
#                         ],
#                         id="wells",
#                         className="pretty_container"
#                     )
#                 ], className="row"),
#                 html.Div([

#                 html.H3('Cases by country'),
#                     dash_table.DataTable(
#                             id='table',
#                             data=df_df_tot_cases.to_dict("rows"),
#                             columns=[{"name": i, "id": i}
#                                      for i in df_tot_cases.columns],
#                             style_table=style_table,
#                             style_cell=style_cell,
#                             style_data_conditional=style_data_conditional,
#                             style_header=style_header,
#                             style_cell_conditional=style_cell_conditional,
#                         )

#         ], className="row"),

#             ], className="two columns"),
#              html.Div([
#                 html.H3('visualisation'),
#                 html.Div([

#                 html.H3('World Map')

#         ], className="row"),

#             ], className="six columns"),

#             html.Div([
#                 html.H3('visualisation'),


#                 html.Div([
#                     html.Div([
#                         html.H3('Total Recoveries'),

#                         ], className="six columns"),

#                     html.Div([
#                         html.H3('Total Deaths'),

#                         ], className="six columns"),
#         ], className="row")

#             ], className="four columns"),
#         ], className="row"),

#         #fOOTER
#          html.Div([
#             html.Div([
#                 html.H3('Footer'),

#             ], className="six columns"),

#             html.Div([
#                 html.H3('Footer'),

#             ], className="six columns"),
#         ], className="row")
#         ])
        
 

    
    
        
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
#     html.Div([
#             html.Div([
#                 html.Div([
#                     html.H1(
#                         children='Table',
#                         style={
#                             'textAlign': 'center',
#                             'color': 'green'
#                         }), dash_table.DataTable(
#                         id='table',
#                         data=df_sum_state.to_dict("rows"),
#                         columns=[{"name": i, "id": i}
#                                  for i in df_sum_state.columns],
#                         style_table=style_table,
#                         style_cell=style_cell,
#                         style_data_conditional=style_data_conditional,
#                         style_header=style_header,
#                         style_cell_conditional=style_cell_conditional,
#                     )
#                 ], className="four columns"),
#                 html.Div([
#                     html.H3('Fires per state in Brazil',
#                             style={
#                                 'textAlign': 'center',
#                                 'color': 'green'}),
#                     dcc.Graph(
#                         id='Graph1',
#                         figure={
#                             'data': [
#                                 {'x': df.state, 'y': df.number, 'type': 'bar', 'name': 'state', 'marker': {'color': [
#                                     '#b50000', '#e63535', '#fa8989'], 'size': 9, 'line': {'width': 1, 'color': 'orange'}}},
#                             ]
#                         }
#                     )
#                 ], className="eight columns"),
#             ], className="row")
#         ])



# if __name__ == '__main__':
#     app.run_server()


In [6]:
# # layout
# app.layout = html.Div([
#       #Band
#      html.Div([
       
#             html.H3('band', style={
#                                 'fontFamily': 'Open Sans',
#                                 'textAlign': 'center',
#                             })
         
#     ], className="row"),
    
#     #VISUALISATIONS
#      html.Div([
#         html.Div([
#             html.H3('visualisation'),
#             html.Div([
       
#                 html.H3('Total confirmed cases')
         
#             ], className="row"),
#             html.Div([
       
#             html.H3('Cases by country')
         
#     ], className="row"),
            
#         ], className="two columns"),
#          html.Div([
#             html.H3('visualisation'),
#             html.Div([
       
#             html.H3('World Map')
         
#     ], className="row"),
            
#         ], className="six columns"),

#         html.Div([
#             html.H3('visualisation'),
            
            
#             html.Div([
#                 html.Div([
#                     html.H3('Total Recoveries'),
                    
#                     ], className="six columns"),

#                 html.Div([
#                     html.H3('Total Deaths'),

#                     ], className="six columns"),
#     ], className="row")
            
#         ], className="four columns"),
#     ], className="row"),
    
#     #fOOTER
#      html.Div([
#         html.Div([
#             html.H3('Footer'),
            
#         ], className="six columns"),

#         html.Div([
#             html.H3('Footer'),
            
#         ], className="six columns"),
#     ], className="row"),
    
# ])






In [7]:
# #header
    
#     html.Div([
#         html.Div([
#             html.H3('Header'),
            
#         ], className="four columns"),
        

#         html.Div([
#             html.H3('header'),
             
#         ], className="four columns"),
        
#         html.Div([
#             html.H3('header'),
            
#         ], className="four columns")
#     ], className="row"),
    
        
#     #NavBar
       
#      html.Div([
#         html.Div([
#             html.H3('nav-bar'),
           
#         ], className="four columns"),
#         html.Div([
#             html.H3('nav-bar'),
           
#         ], className="four columns"),

#         html.Div([
#             html.H3('nav-bar'),
            
#         ], className="four columns"),
#     ], className="row"),
    
#       #Band
#      html.Div([
       
#             html.H3('band')
         
#     ], className="row"),
    
#     #VISUALISATIONS
#      html.Div([
#         html.Div([
#             html.H3('visualisation'),
#             html.Div([
       
#                 html.H3('Total confirmed cases')
         
#             ], className="row"),
#             html.Div([
       
#             html.H3('Cases by country')
         
#     ], className="row"),
            
#         ], className="two columns"),
#          html.Div([
#             html.H3('visualisation'),
#             html.Div([
       
#             html.H3('World Map')
         
#     ], className="row"),
            
#         ], className="six columns"),

#         html.Div([
#             html.H3('visualisation'),
            
            
#             html.Div([
#                 html.Div([
#                     html.H3('Total Recoveries'),
                    
#                     ], className="six columns"),

#                 html.Div([
#                     html.H3('Total Deaths'),

#                     ], className="six columns"),
#     ], className="row")
            
#         ], className="four columns"),
#     ], className="row"),
    
#     #fOOTER
#      html.Div([
#         html.Div([
#             html.H3('Footer'),
            
#         ], className="six columns"),

#         html.Div([
#             html.H3('Footer'),
            
#         ], className="six columns"),
#     ], className="row"),


### Dashboard pages template

In [8]:
# # Style sheet
# external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']

# # App
# app = dash.Dash(__name__, external_stylesheets=external_stylesheets)

# # layout
# app.layout = html.Div([
    
#     #header
    
#     html.Div([
#         html.Div([
#             html.H3('Header'),
            
#         ], className="four columns"),
        

#         html.Div([
#             html.H3('header'),
             
#         ], className="four columns"),
        
#         html.Div([
#             html.H3('header'),
            
#         ], className="four columns")
#     ], className="row"),
    
        
#     #NavBar
       
#      html.Div([
#         html.Div([
#             html.H3('nav-bar'),
           
#         ], className="four columns"),
#         html.Div([
#             html.H3('nav-bar'),
           
#         ], className="four columns"),

#         html.Div([
#             html.H3('nav-bar'),
            
#         ], className="four columns"),
#     ], className="row"),
    
#       #Band
#      html.Div([
       
#             html.H3('band')
         
#     ], className="row"),
    
#     #VISUALISATIONS
#      html.Div([
#         html.Div([
#             html.H3('visualisation'),
#             html.Div([
       
#                 html.H3('Total confirmed cases')
         
#             ], className="row"),
#             html.Div([
       
#             html.H3('Cases by country')
         
#     ], className="row"),
            
#         ], className="two columns"),
#          html.Div([
#             html.H3('visualisation'),
#             html.Div([
       
#             html.H3('World Map')
         
#     ], className="row"),
            
#         ], className="six columns"),

#         html.Div([
#             html.H3('visualisation'),
            
            
#             html.Div([
#                 html.Div([
#                     html.H3('Total Recoveries'),
                    
#                     ], className="six columns"),

#                 html.Div([
#                     html.H3('Total Deaths'),

#                     ], className="six columns"),
#     ], className="row")
            
#         ], className="four columns"),
#     ], className="row"),
    
#     #fOOTER
#      html.Div([
#         html.Div([
#             html.H3('Footer'),
            
#         ], className="six columns"),

#         html.Div([
#             html.H3('Footer'),
            
#         ], className="six columns"),
#     ], className="row"),
    
# ])

# if __name__ == '__main__':
#     app.run_server()


In [9]:
import plotly.graph_objects as go

fig1 = go.Figure(go.Indicator(
    
    value = 200,
    delta = {'reference': 160},
    gauge = {
        'axis': {'visible': False}},
    domain = {'row': 0, 'column': 0}))

fig1 = fig1.update_layout(
    grid = {'rows': 5, 'columns': 5, 'pattern': "independent"},
    template = {'data' : {'indicator': [{
        'title': {'text': "Total Cases"},
        'mode' : "number+delta+gauge",
        'delta' : {'reference': 90}}]
                         }})
fig1.show()

In [10]:
#Style sheet
external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']

# App
app = dash.Dash(__name__, external_stylesheets=external_stylesheets)

app.layout = html.Div(children=[
        #header
    
    html.Div([
        html.Div([
            html.Img(src="https://dcassetcdn.com/design_img/3666062/781819/781819_21156990_3666062_e336b60a_image.png",className='four columns')
            
        ], className="four columns"),
        

        html.Div([
            html.H3('Latest Covid 19 Update', style={
                                'fontFamily': 'Open Sans',
                                'textAlign': 'center',
                            }),
             
        ], className="four columns"),
        
        html.Div([
            html.H3('#StayAtHome', style={
                                'fontFamily': 'Open Sans',
                                'textAlign': 'right',
                            }),
            
        ], className="four columns")
    ], className="row"),
    
    dcc.Tabs(id='tabs-example', value='tab', children=[
        dcc.Tab(label='Global Map', value='tab-1'),
        dcc.Tab(label='South African cases', value='tab-2'),
        dcc.Tab(label='Analysis', value='tab-3'),
    ]),
    html.Div(id='tabs-example-content')
])

In [11]:
@app.callback(Output('tabs-example-content', 'children'),
              [Input('tabs-example', 'value')])
def render_content(tab):
    
    if tab == 'tab-1':
        return html.Div([
            html.Div([
                dcc.Graph(
                                    
                                        id='figure',
                                        figure=
                                          go.Figure(fig1)
                                    ),
                
                #VISUALISATIONS
     
                html.Div([
                    dcc.Graph(
                                    
                                        id='figure',
                                        figure=
                                          go.Figure(fig1)
                                    ),
                    html.H3('visualisation', style={
                                'fontFamily': 'Open Sans',
                                'textAlign': 'center',
                            }),
                    html.Div([
                        
                        html.Div(
                            [
                                  dcc.Graph(
                                    
                                        id='figure',
                                        figure=
                                          go.Figure(fig1)
                                    ),
                                
                            ])
                    ], className="row"),
                    html.Div([

                    html.H3('Cases by country')

            ], className="row"),

                ], className="two columns"),
                 html.Div([
                    html.H3('visualisation',style={
                                'fontFamily': 'Open Sans',
                                'textAlign': 'center',
                            }),
                    html.Div([

                    html.H3('World Map',style={
                                'fontFamily': 'Open Sans',
                                'textAlign': 'center',
                            })

            ], className="row"),

                ], className="six columns"),

                html.Div([
                    html.H3('visualisation',style={
                                'fontFamily': 'Open Sans',
                                'textAlign': 'center',
                            }),


                    html.Div([
                        html.Div([
                            html.H3('Total Recoveries'),

                            ], className="six columns"),

                        html.Div([
                            html.H3('Total Deaths'),

                            ], className="six columns"),
            ], className="row")

                ], className="four columns"),
            ], className="row"),

            #fOOTER
             html.Div([
                html.Div([
                    html.H3('Footer'),

                ], className="six columns"),

                html.Div([
                    html.H3('Footer'),

                ], className="six columns"),
            ], className="row"),
        ])

    elif tab == 'tab-2':   
        return html.P("You got it T")
    else:
        return html.H1('O ntja saan!!!')
    


# if __name__ == '__main__':
#     app.run_server()

In [12]:
width=800, height=400

SyntaxError: can't assign to literal (<ipython-input-12-ea9379c0e959>, line 1)